In [87]:
#Run routeLinkedList.ipynb
%run routeLinkedList.ipynb

In [78]:
REQUESTSTATES = {
    "WAITING":0,
    "SCHEDULED":1,
    "ONGOING":2,
    "PREDICTED":3,
    "COMPLETED":4,
    "REJECTED":5
    }

VEHICLESTATES = {
    "IDLE":0,
    "MOVING":1,
    "MOVINGFULL":2
}

In [79]:
class RequestCords:
    def __init__(self, latidude, longitude, start):
        self.latidude = latidude
        self.longitude = longitude
        self.start = start

    def __str__(self):
        return "lat: "+ str(self.latidude) + ",long: " + str(self.longitude) + ", " + str(self.start)

In [80]:
class Request:
    def __init__(self, passengerAmount, origin, destination, time):
        self.request_id = id(self)
        self.passengerAmount = passengerAmount
        self.origin = origin
        self.destination = destination
        self.timeOfRequest = time

    def __str__(self):
        return "Request: id: {}, passengerAmount: {}, origin: {}, destination: {}, time: {}".format(self.request_id, self.passengerAmount, self.origin, self.destination, self.timeOfRequest)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.request_id == other.request_id
    
    #Change state of request
    def changeState(self, state):
        self.state = REQUESTSTATES[state]

    #Getters
    def getId(self):
        return self.request_id
    
    def getPassengerAmount(self):
        return self.passengerAmount
    
    def getOrigin(self):
        return self.origin
    
    def getDestination(self):
        return self.destination
    
    def getTime(self):
        return self.timeOfRequest
    
    def getState(self):
        return self.state

In [81]:
class Routes:
    def __init__(self):
        self.routeList = RouteLinkedList()

    def handleAddRequest(self, request, indexOrigin, indexDestination):
        if indexDestination < indexOrigin:
            raise ValueError("Tried to add a request to the routeList, but the destination happened before the pickup")
        self.routeList.insertAtIndex(request.origin, request.request_id, indexOrigin)
        self.routeList.insertAtIndex(request.destination, request.request_id, indexDestination)

    def handleNextArrival(self):
        cordPoint = self.routeList.deleteAtStart()
        if cordPoint == None:
            print("Tried to handle an arrival of a request, but there were no requests in the routeList")
            return None
        else:
            return cordPoint
    
    def __str__(self):
        return str(self.routeList)

    def __repr__(self):
        return self.__str__()
    
    def getSize(self):
        return self.routeList.getSize()

In [82]:
class Vehicle:
    def __init__(self, capacity, origin):
        self.vehicle_id = id(self)
        self.capacity = capacity
        self.currentAmountOfPassengers = 0
        self.currentPosition = origin
        self.state = VEHICLESTATES["IDLE"]
        self.route = Routes()
        self.onGoingRequestList = {}
        self.completedRequestList = {}

    def __str__(self):
        return "Vehicle: id: {}, capacity: {}, currentPosition: {}, route: {}".format(self.vehicle_id, self.capacity, self.currentPosition, self.route.getSize())
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.vehicle_id == other.vehicle_id
    
    def addRequestToRoute(self, request, indexOrigin, indexDestination):
    
        #Check Passengers Limit
        if request.getPassengerAmount() + self.currentAmountOfPassengers > self.capacity:
            print("Request exceeds capacity of vehicle")
        else:
            #If vehicle has space add request to the route at the specified indexes
            self.route.handleAddRequest(request, indexOrigin, indexDestination)
            request.changeState("SCHEDULED")
            self.onGoingRequestList[request.getId()] = request
            self.currentAmountOfPassengers += request.getPassengerAmount()
    
    def arrivedAtNextStop(self):
        cordPoint = self.route.handleNextArrival()
        if not cordPoint.cords.start:
            #Meaning a request has been completed
            completedRequest = self.onGoingRequestList[cordPoint.requestId]
            self.currentPosition = cordPoint.cords
            self.currentAmountOfPassengers -= completedRequest.getPassengerAmount()
        else:
            #Meaning a request has been picked up
            self.onGoingRequestList[cordPoint.requestId].changeState("ONGOING")
            self.currentPosition = cordPoint.cords

    def adjustRoutes():
        #Need to be implemented that will allow adjustment of routes occuring to the new requests
        pass


    def printRoute(self):
        return self.route